In [1]:
import pandas as pd, json, requests

In [2]:
with open("hide/personnal-keys.json") as f : 
    ACCESS_TOKEN = json.load(f)["ZENODO_KEY"]
    

### memo documentation en ligne  Zenodo

le guide pour la recherche avancée https://help.zenodo.org/guides/search/

guide utilisation API https://developers.zenodo.org/#representation

In [22]:
## champs pour date created, publication_date, updated_date
res = requests.get(
    "https://zenodo.org/api/records",
     params = {
         "q" : "publication_date:[2020-01-01 TO 2020-12-31]",
         "size" : 300,
         "sort" : "mostrecent",
         "all_version" : False,
         "access_tpoken" : ACCESS_TOKEN
                }
)

In [23]:
res.url

'https://zenodo.org/api/records/?q=publication_date%3A%5B2020-01-01+TO+2020-12-31%5D&size=300&sort=mostrecent&all_version=False&access_tpoken=4A1P6e4gIkvhpcNpVpI05C9yVCjJIDD3vljILoJ1wlyxT9VA70ZXyPMdjqHB'

In [24]:
res_json = res.json()

In [6]:
with open("zenodo_raw.json", 'w') as f : 
    json.dump(res_json, f)

## Documentation

###  file_type

ce n'est apparemment pas un voca contrôlé (comme pour les mots clés), donc beaucoup de fichier sont sans filetye. on suppose que sont affiché seuls les 10 premiers. 

In [25]:
## repartition par file_tpe
res_json["aggregations"]["file_type"]

{'buckets': [{'doc_count': 82856, 'key': 'pdf'},
  {'doc_count': 33490, 'key': 'png'},
  {'doc_count': 32539, 'key': 'html'},
  {'doc_count': 19802, 'key': 'zip'},
  {'doc_count': 11625, 'key': 'jpg'},
  {'doc_count': 5393, 'key': 'xlsx'},
  {'doc_count': 4295, 'key': 'xml'},
  {'doc_count': 4276, 'key': 'csv'},
  {'doc_count': 3556, 'key': 'docx'},
  {'doc_count': 3508, 'key': 'txt'}],
 'doc_count_error_upper_bound': 0,
 'sum_other_doc_count': 26739}

In [8]:
ftype = pd.DataFrame.from_dict(res_json["aggregations"]["file_type"]["buckets"])
ftype

,doc_count,key
0,82851,pdf
1,33485,png
2,32539,html
3,19807,zip
4,11625,jpg
5,5393,xlsx
6,4295,xml
7,4278,csv
8,3556,docx
9,3509,txt


In [9]:
ftype["doc_count"].sum()

201338

### doc_type

In [10]:
## repartition par doc_type
res_json["aggregations"]["type"]["buckets"] ## c'est une liste très longyue car les doctype on des sous types

[{'doc_count': 114871,
  'key': 'publication',
  'subtype': {'buckets': [{'doc_count': 61464, 'key': 'article'},
    {'doc_count': 32464, 'key': 'taxonomictreatment'},
    {'doc_count': 5719, 'key': 'conferencepaper'},
    {'doc_count': 4629, 'key': 'other'},
    {'doc_count': 2201, 'key': 'report'},
    {'doc_count': 2110, 'key': 'deliverable'},
    {'doc_count': 1502, 'key': 'book'},
    {'doc_count': 1312, 'key': 'section'},
    {'doc_count': 1152, 'key': 'preprint'},
    {'doc_count': 912, 'key': 'workingpaper'},
    {'doc_count': 725, 'key': 'thesis'},
    {'doc_count': 290, 'key': 'technicalnote'},
    {'doc_count': 97, 'key': 'proposal'},
    {'doc_count': 77, 'key': 'milestone'},
    {'doc_count': 70, 'key': 'softwaredocumentation'},
    {'doc_count': 55, 'key': 'patent'},
    {'doc_count': 47, 'key': 'datamanagementplan'},
    {'doc_count': 34, 'key': 'annotationcollection'},
    {'doc_count': 11, 'key': 'peerreview'}],
   'doc_count_error_upper_bound': 0,
   'sum_other_doc_co

In [11]:
### contenu d'un doctype
res_json["aggregations"]["type"]["buckets"][1]

{'doc_count': 45113,
 'key': 'image',
 'subtype': {'buckets': [{'doc_count': 43279, 'key': 'figure'},
   {'doc_count': 886, 'key': 'photo'},
   {'doc_count': 775, 'key': 'drawing'},
   {'doc_count': 84, 'key': 'other'},
   {'doc_count': 48, 'key': 'plot'},
   {'doc_count': 41, 'key': 'diagram'}],
  'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0}}

In [29]:
### les trois clés qui nous intéressent sont "doc_count", "key"
### creer un dictionnaire pour récupérer ces infos
doctype = {}
keys = ["doc_count", "key"]

In [30]:
## iterer sur tous les éléments de la liste
for elem in res_json["aggregations"]["type"]["buckets"] :
    
    doctype.update(
    {elem["key"] : elem["doc_count"]}
    )


In [37]:
doctype

{'publication': 114877,
 'image': 45118,
 'dataset': 23387,
 'software': 13281,
 'presentation': 4986,
 'other': 2741,
 'poster': 2089,
 'video': 1421,
 'lesson': 1007,
 'physicalobject': 19}

In [41]:
df = pd.DataFrame.from_dict(doctype, orient = "index").T
df

,publication,image,dataset,software,presentation,other,poster,video,lesson,physicalobject
0,114877,45118,23387,13281,4986,2741,2089,1421,1007,19


In [18]:
df.to_csv("distribution_doctype.csv", index = False)

In [15]:
"total de doc dans les doctype " + str(df["doc_count"].sum())

'total de doc dans les doctype 208923'

In [16]:
"total de doc présents dans les hits " + str(res_json["hits"]["total"])

'total de doc présents dans les hits 208925'